#Environment

In [1]:
import gym
import numpy as np

#
# Environment
#
env = gym.make('CartPole-v1')
state = env.reset()
action = env.action_space.sample()

print('State space: ', env.observation_space)
print('Initial state: ', state)
print('\nAction space: ', env.action_space)
print('Random action: ', action)

State space:  Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
Initial state:  [ 0.02685345  0.0105703  -0.02593711  0.00075185]

Action space:  Discrete(2)
Random action:  0


#DQN modelling

In [2]:
# DQN Modeling
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

num_state = env.observation_space.shape[0]
num_action = env.action_space.n


model = Sequential()
# Hidden layer with 512 nodes
model.add(Dense(512, input_dim= num_state, kernel_initializer='he_uniform', activation='relu'))
# Hidden layer with 256 nodes
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# Hidden layer with 64 nodes
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(num_action, activation='linear'))
model.compile(loss='mse', optimizer="adam")

#Learning process

In [3]:
import random
from collections import deque
from tqdm import tqdm

num_episode = 300
memory = deque(maxlen=2000)

# Hyper parameter
epsilon = 0.3
gamma = 0.95
batch_size = 32

# DQN Learning
for episode in tqdm(range(num_episode)):
    state = env.reset()
    done = False
    while not done:
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            q_value = model.predict(state.reshape(1, num_state))
            action = np.argmax(q_value[0])
        next_state, reward, done, info = env.step(action)
        # Memory
        memory.append((state, action, reward, next_state, done))
        
        state = next_state
    
    # Replay
    if len(memory) > batch_size:
        mini_batch = random.sample(memory, batch_size)
        for state, action, reward, next_state, done in mini_batch:
            if done:
                target = reward
            else:
                target = reward + gamma * (np.max(model.predict(next_state.reshape(1, num_state))[0]))
            q_value = model.predict(state.reshape(1, num_state))
            q_value[0][action] = target
            model.fit(state.reshape(1, num_state), q_value, epochs=1, verbose=0)

env.close()

100%|██████████| 300/300 [17:26<00:00,  3.49s/it]


#Save Model

In [4]:
import os

save_dir = os.getcwd()
model_name = 'keras_dqn_trained_model.h5'

# Save model and weights
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

##Install for visualization

In [5]:
!pip install gym pyvirtualdisplay

In [6]:
!apt-get install xvfb python-opengl ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl xvfb
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,280 kB of archives.
After this operation, 7,682 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.7 [783 kB]
Fetched 1,280 kB in 0s (12.1 MB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 144628 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Selecting previously unselected package xvfb.
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.7_amd6

In [7]:
from gym.wrappers import Monitor

import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video(file_infix):
    with open('./video/openaigym.video.%s.video000000.mp4' % file_infix, 'r+b') as f:
        video = f.read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="Trained CartPole" autoplay 
                loop style="height: 200px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    
def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

## load the model that we saved

In [8]:
from  tensorflow.keras.models import load_model
import os

load_dir = os.getcwd()
model_name = 'keras_dqn_trained_model.h5'
model_path = os.path.join(load_dir, model_name)
model = load_model(model_path)

In [9]:
import gym
import numpy as np

num_state = env.observation_space.shape[0]
env = wrap_env(gym.make('CartPole-v1'))
state = env.reset()
done = False

mem = []

while not done:
    state = np.array(state).reshape(1, num_state)
    q_value = model.predict(state)
    mem.append(q_value[0])
    action = np.argmax(q_value[0])
    state, reward, done, info = env.step(action)

file_infix = env.file_infix
env.close()

show_video(file_infix)